<a href="https://colab.research.google.com/github/romapavelko01/NLP_SDLC_project/blob/classifications/Classification_MB_for_top10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import pandas as pd
import numpy as np

In [27]:
filename = "/content/drive/MyDrive/SDLC/news_analysis_project/data/final_news_category_dataset.json"
df = pd.read_json(filename, orient='split')
df.head(3)

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26


In [28]:
df = df[df.category.isin(df['category'].value_counts().sort_values(ascending=False).index[:10])]
print(len(df), sum(df['category'].value_counts().sort_values(ascending=False).values[:10]))
df.head(3)

120008 120008


,category,headline,authors,link,short_description,date
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26


In [32]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Text preprocessing

In [29]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
import re
import string


def cleaning_function(sentence):
    """
    Function needed to perform data preprocessing: removing punctuation symbols,
    stop_words and other random things in order to obtain clean text
    """
    # the following line removes numbers from text
    result = re.sub(r'\d+', '', sentence.lower())

    # the following line removes any punctuation from the text
    result = result.translate(str.maketrans('','',string.punctuation))
    return [word for word in result.split() if not word in stop_words]


def collect_words(lst):
    """
    Given a list of lists of words,
    return a list of all the words in a list representation of the text.
    """
    return [word for sentence in lst for word in sentence]

# Classification based on short description

In [31]:
df_1 = df.copy()
df_1['processed_description'] = df_1['short_description'].apply(lambda x: ' '.join(cleaning_function(x)))
df_1 = df_1[['category', 'processed_description']]
df_1.head()

,category,processed_description
1,ENTERTAINMENT,course song
2,ENTERTAINMENT,actor longtime girlfriend anna eberstein tied ...
3,ENTERTAINMENT,actor gives dems asskicking fighting hard enou...
4,ENTERTAINMENT,dietland actress said using bags really cathar...
5,ENTERTAINMENT,right equate horrific incidents sexual assault...


## Without PCA

### Using Countvectorizer




In [33]:
cv_u = CountVectorizer()
X_u = cv_u.fit_transform(df_1.processed_description)

X_train, X_test, y_train, y_test = train_test_split(X_u, df_1['category'].values, test_size=0.2)

classifier_u = MultinomialNB()
classifier_u.fit(X_train, y_train)
y_pred = classifier_u.predict(X_test)

accuracy_score(y_test, y_pred)

0.6245729522539788

### Using Tfidfvectorizer

In [34]:
tf_u = TfidfVectorizer()
X_u = tf_u.fit_transform(df_1.processed_description)

X_train, X_test, y_train, y_test = train_test_split(X_u, df_1['category'].values, test_size=0.2)

classifier_u = MultinomialNB()
classifier_u.fit(X_train, y_train)
y_pred = classifier_u.predict(X_test)

accuracy_score(y_test, y_pred)

0.5327056078660112

# Classification based on headline

In [35]:
new_df = df.copy()
new_df['processed_headline'] = new_df['headline'].apply(lambda x: ' '.join(cleaning_function(x)))
new_df = new_df[['category', 'processed_headline']]
new_df.head()

,category,processed_headline
1,ENTERTAINMENT,smith joins diplo nicky jam world cups officia...
2,ENTERTAINMENT,hugh grant marries first time age
3,ENTERTAINMENT,jim carrey blasts castrato adam schiff democra...
4,ENTERTAINMENT,julianna margulies uses donald trump poop bags...
5,ENTERTAINMENT,morgan freeman devastated sexual harassment cl...


## Without PCA

### Using Countvectorizer

In [36]:
cv_u = CountVectorizer()
X_u = cv_u.fit_transform(new_df.processed_headline)

X_train, X_test, y_train, y_test = train_test_split(X_u, new_df['category'].values, test_size=0.2)

classifier_u = MultinomialNB()
classifier_u.fit(X_train, y_train)
y_pred = classifier_u.predict(X_test)

accuracy_score(y_test, y_pred)

0.7362719773352221

### Using Tfidfvectorizer

In [37]:
tf_u = TfidfVectorizer()
X_u = tf_u.fit_transform(new_df.processed_headline)

X_train, X_test, y_train, y_test = train_test_split(X_u, new_df['category'].values, test_size=0.2)

classifier_u = MultinomialNB()
classifier_u.fit(X_train, y_train)
y_pred = classifier_u.predict(X_test)

accuracy_score(y_test, y_pred)

0.6589450879093409

# Parameter tuning

In [38]:
from sklearn.model_selection import GridSearchCV

## For short description

In [39]:
params = {'alpha': [0.01, 0.1, 1, 5],
          'fit_prior': [False, True]}

X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_u, df_1['category'].values, test_size=0.2)

In [40]:
mnb_d = GridSearchCV(MultinomialNB(), param_grid=params, n_jobs=-1)
mnb_d.fit(X_train_d, y_train_d)

GridSearchCV(estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.1, 1, 5],
                         'fit_prior': [False, True]})

In [41]:
mnb_d.best_params_

{'alpha': 1, 'fit_prior': False}

In [42]:
y_pred_best_d = mnb_d.predict(X_test_d)
accuracy_score(y_test_d, y_pred_best_d)

0.738480126656112

## For headline

In [43]:
cv_u_h = CountVectorizer()
X_u_h = cv_u.fit_transform(new_df.processed_headline)

X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(X_u_h, new_df['category'].values, test_size=0.2)

In [44]:
mnb_h = GridSearchCV(MultinomialNB(), param_grid=params, n_jobs=-1)
mnb_h.fit(X_train_h, y_train_h)

GridSearchCV(estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.1, 1, 5],
                         'fit_prior': [False, True]})

In [45]:
mnb_h.best_params_

{'alpha': 1, 'fit_prior': False}

In [46]:
y_pred_best_h = mnb_d.predict(X_test_h)
accuracy_score(y_test_h, y_pred_best_h)

0.781101574868761